In [ ]:
import requests
import xml.etree.ElementTree as ET

def extract_information(xml_url):
    response = requests.get(xml_url)

    if response.status_code == 200:
        xml_data = response.content
        root = None
        try:
            root = ET.fromstring(xml_data)
        except ET.ParseError as e:
            print(f"Error: Parsing XML failed for URL {xml_url}")
            print(f"Exception: {e}")

        extracted_data = None
    
        if root is not None:
            extracted_data = []
            # Find all <kind> and <value> elements
            kind_elements = root.findall(".//kind")
            value_elements = root.findall(".//value")

            if len(kind_elements) == len(value_elements):
                for kind_element, value_element in zip(kind_elements, value_elements):
                    kind = kind_element.text
                    value = value_element.text

                    extracted_data.append({
                        'kind': kind,
                        'value': value,
                    })

    
            else:
                print("Error: Number of <kind> elements does not match the number of <value> elements in the XML.")
                return None
            
        return extracted_data
    
    else:
 #       print(f"Error: Unable to fetch data from {xml_url}")
        return None



def format_number_with_zeroes(number, fixed_length):
    # Convert the number to a string and use zfill to add leading zeroes
    formatted_number = str(number).zfill(fixed_length)
    return formatted_number

smiles = []
melting_pts = []


In [ ]:

if __name__ == "__main__":
    for i in range(0,5000):

        if i % 1000 == 0:
            print(i)

        formatted_i = format_number_with_zeroes(i, fixed_length=7)
        xml_url = "https://np-mrd.org/natural_products/NP%s.xml" %(formatted_i)
        extracted_data = extract_information(xml_url)

        melting_found = False

        if extracted_data:
            for data in extracted_data:
#                print("Kind:", data['kind'])
#                print("Value:", data['value'])
#                print("-" * 20)
                    
                if data['kind'] == 'logs':
                    melting_found = True
                    melting_pts.append(data['value'])
                if data['kind'] == 'smiles' and melting_found == True:
                    smiles.append(data['value'])



In [ ]:
print(melting_pts)
print(smiles)
print(len(melting_pts),len(smiles))

In [ ]:
import pandas as pd 

vector1 = pd.Series(melting_pts)
vector2 = pd.Series(smiles)

matrix_data = pd.concat([vector1, vector2], axis=1)
# Save the DataFrame to a CSV file named 'matrix_data.csv'

matrix_data.to_csv('matrix_data1904.csv', index=False)